In [37]:
import pandas as pd

# Load the dataset
df = pd.read_csv('input_game.csv')

# Check for missing values
print(df.isnull().sum())

game_id      0
p1_id        0
p2_id        0
p1_action    0
p2_action    0
turn         0
dtype: int64


In [38]:
# Convert 'TRUST' to 0 and 'CHEAT' to 1
df['p1_action'] = df['p1_action'].apply(lambda x: 0 if x == 'TRUST' else 1)
df['p2_action'] = df['p2_action'].apply(lambda x: 0 if x == 'TRUST' else 1)

# Number of times the opponent cheated in the last 4 moves
df['opp_cheat_last_4'] = df.groupby(['game_id', 'p1_id'])['p2_action'].transform(lambda x: x.rolling(4).sum().fillna(0).astype(int))

# Number of times the player trusted in the last 4 moves
df['self_trust_last_4'] = df.groupby(['game_id', 'p1_id'])['p1_action'].transform(lambda x: x.rolling(4).sum().fillna(0).astype(int))


In [39]:
from sklearn.ensemble import RandomForestClassifier

# Define the features and target
X = df[['opp_cheat_last_4', 'self_trust_last_4']]
y = df['p1_action']

# Initialize the model
model = RandomForestClassifier()

# Train the model
model.fit(X, y)


/Users/sumitprakash/venv-metal/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/sumitprakash/venv-metal/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/sumitprakash/venv-metal/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/sumitprakash/venv-metal/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future

RandomForestClassifier()

In [40]:
# Predict the moves
df['predicted_action'] = model.predict(X)



/Users/sumitprakash/venv-metal/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/sumitprakash/venv-metal/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/sumitprakash/venv-metal/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [42]:
df.head()

,game_id,p1_id,p2_id,p1_action,p2_action,turn,opp_cheat_last_4,self_trust_last_4,predicted_action
0,0,174,125,0,1,1,0,0,0
1,0,174,125,1,0,2,0,0,0
2,0,174,125,0,0,3,0,0,0
3,0,174,125,0,0,4,1,1,0
4,0,174,125,0,0,5,0,1,0


In [48]:
# Identify the strategies
a = df.groupby(['game_id', 'p1_id']).apply(lambda x: (x['p1_action'] == x['predicted_action']).all())

KeyError: 'p1_action'

In [44]:
a = a.reset_index(drop=True)

In [45]:
df['strategy'] = a

In [46]:
# Count the number of distinct strategies
N = df['strategy'].nunique()
print(N)

2


In [47]:
df.to_csv('x.csv', index=True)